# Package Import

In [1]:
import pandas as pd 
import numpy as np

### Renaming

In [2]:
rename_dict = {
    'ISBN': 'isbn',
    'Book-Author': 'authors',
    'Book-Title': 'title',
    'Year-Of-Publication': 'original_publication_year',
    'Image-URL-S': 'small_image_url'
}

ratings_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\ratings.csv", sep = ';', encoding = 'latin-1')
users_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\users.csv", sep = ';', encoding = 'latin-1')
books_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\books.csv", sep = ';', encoding = 'latin-1', error_bad_lines=False)
books_l = books_l.rename(columns = rename_dict).drop(['Publisher', 'Image-URL-M','Image-URL-L'], axis = 1)
books_l['original_publication_year'] =  pd.to_numeric(books_l['original_publication_year'], errors='coerce', downcast='integer')

# Import smaller set
ratings_s = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\goodbooks-10k\ratings_10k.csv")
books_s = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\goodbooks-10k\books_10k.csv")
books_s_redux = books_s[['isbn','authors','title','original_publication_year','small_image_url']]
books_s_redux['original_publication_year'] = pd.to_numeric(books_s['original_publication_year'], errors='coerce', downcast='integer')

books_df = pd.concat([books_l, books_s_redux], axis = 0)
books_df = books_df.drop_duplicates(subset = 'isbn')

C:\Users\penna\AppData\Local\Temp\ipykernel_20660\2935986721.py:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books_l = pd.read_csv(r"C:\Users\penna\Documents\Porta\Education\Coding\librec\librec\data_warehouse\larger_set\books.csv", sep = ';', encoding = 'latin-1', error_bad_lines=False)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: exp

In [3]:
rating_column_dict = {
    'User-ID': 'user_id_adj',
    'ISBN': 'isbn',
    'Book-Rating': 'rating_adj'
}

ratings_s['isbn'] = ratings_s['book_id'].map(books_s[['id', 'isbn']].set_index('id').to_dict()['isbn'])
ratings_s['rating_adj'] = ratings_s['rating'] * 2
ratings_s['user_id_adj'] = ratings_s['user_id'] + ratings_l['User-ID'].max() + 1
ratings_s_redux = ratings_s[['user_id_adj', 'isbn', 'rating_adj']]
ratings_l = ratings_l.rename(columns = rating_column_dict)
ratings_df = pd.concat([ratings_l, ratings_s_redux], axis = 0)



In [4]:
ratings_s.shape

(981756, 6)

In [5]:
ratings_df

,user_id_adj,isbn,rating_adj
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
981751,327241,375700455,10
981752,327862,375700455,8
981753,328238,375700455,10
981754,328979,375700455,10


In [12]:
# create a pivot table of users, with isbn as columns and ratings as values

ratings_pivot = ratings_df.iloc[0:100000].pivot_table(index = 'user_id_adj', columns = 'isbn', values = 'rating_adj', fill_value= 0).astype('Sparse[int]')

In [11]:
ratings_pivot

isbn,0002005018,0002231115,0002232766,0002240114,000225669X,000254794,0002558122,0002740230,0006128831,0006379702,...,9782264014184,9782922145441,9788401499883,9871138016,9995585227,B0000BLD7X,B158991965,DITISEENSOORT,N3453124715,O6712345670
user_id_adj,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
len(ratings_df.user_id_adj.unique())

158707

In [18]:
ratings_df = ratings_df.dropna()
len(ratings_df)

2064314

In [20]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

person_u = list(ratings_df.user_id_adj.unique())
thing_u = list(ratings_df.isbn.unique())

data = ratings_df['rating_adj'].tolist()
cat_type_col = CategoricalDtype(categories=thing_u, ordered=True)
cat_type_row = CategoricalDtype(categories=person_u, ordered=True)
row = ratings_df.user_id_adj.astype(cat_type_row).cat.codes
col = ratings_df.isbn.astype(cat_type_col).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(person_u), len(thing_u)))

MemoryError: Unable to allocate 205. GiB for an array with shape (157746, 349400) and data type int32